In [1]:
%load_ext autoreload
%autoreload 2
import random
import yaml
import os
import logging
import matplotlib.pyplot as plt
import cv2

import scipy.io as io
import torch
import torch.optim
import torch.utils.data
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import open3d as o3d

from tensorboardX import SummaryWriter
from utils.utils import getWriterPath
from settings import EXPER_PATH
from utils.loader import dataLoader, modelLoader, pretrainedLoader
from utils.logging import *
from copy import deepcopy as dc
from utils.d2s import DepthToSpace, SpaceToDepth
from train_cubemap import *
from utils.utils import flattenDetection
from Train_model_frontend_cubemap import thd_img
from utils_custom import *
from PIL import Image
import matplotlib.pyplot as plt

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
def measure_t(start, before):
    now = time.time()
    runt = now-start
    h = runt//3600
    m = (runt - h*3600)//60
    s = round(runt - h*3600 - m*60)
    prtstr = f"iter {_iter+1} in {len(train_loader)}, \
            {round((_iter+1)/len(train_loader), 4) * 100}%\
            {h}h {m}m {s}s"
    runt2 = before-start
    h2 = runt2//3600
    m2 = (runt2 - h2*3600)//60
    s2 = round(runt2 - h2*3600 - m2*60)
    prtstr += f"  ({h2}h {m2}m {s2}s)"
    return now  
args = Namespace(command='train_joint', config='configs/magicpoint_cubemap.yaml', debug=False, eval=False, exper_name='cubemap_dataset', func=train_joint)


from utils_custom import *
from utils_custom_visualize import *

with open(args.config, 'r') as f:
    config = yaml.safe_load(f)
EXPER_PATH = 'logs'
output_dir = EXPER_PATH
output_dir

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


'logs'

In [4]:
############################################### train_joint

torch.set_default_tensor_type(torch.FloatTensor)
task = config['data']['dataset']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info('train on device: %s', device)
with open(os.path.join(output_dir, 'config.yml'), 'w') as f:
    yaml.dump(config, f, default_flow_style=False)
# writer = SummaryWriter(getWriterPath(task=args.command, date=True))
writer = SummaryWriter(getWriterPath(task=args.command, 
    exper_name=args.exper_name, date=True))
## save data
save_path = get_save_path(output_dir)

############################################### 
data = dataLoader(config, dataset=task, warp_input=True)
train_loader, val_loader = data['train_loader'], data['val_loader']

datasize(train_loader, config, tag='train')
datasize(val_loader, config, tag='val')
# init the training agent using config file
# from train_model_frontend import Train_model_frontend
from utils.loader import get_module
train_model_frontend = get_module('', config['front_end_model'])
train_agent = train_model_frontend(config, save_path=save_path, device=device)

# writer from tensorboard
train_agent.writer = writer

# feed the data into the agent
train_agent.train_loader = train_loader
train_agent.val_loader = val_loader

# load model initiates the model and load the pretrained model (if any)
train_agent.loadModel()
train_agent.dataParallel()


2023-04-10 16:23:18 koala-A520M-H root[25190] INFO train on device: cuda
2023-04-10 16:23:18 koala-A520M-H root[25190] INFO => will save everything to logs/checkpoints
2023-04-10 16:23:18 koala-A520M-H root[25190] INFO workers_train: 1, workers_val: 1
2023-04-10 16:23:18 koala-A520M-H root[25190] INFO == train split size 2160 in 2160 batches
2023-04-10 16:23:18 koala-A520M-H root[25190] INFO == val split size 2160 in 2160 batches
2023-04-10 16:23:18 koala-A520M-H root[25190] INFO => creating model: SuperPointNet_cubemap


dataset: CubemapDataset
not_warped_images:  True
not_warped_images:  True
Load Train_model_frontend!!
check config!! {'train_iter': 200000, 'save_interval': 2000, 'tensorboard_interval': 1000, 'model': {'subpixel': {'enable': False}, 'train_only_descriptor': True, 'name': 'SuperPointNet_cubemap', 'params': {}, 'detector_loss': {'loss_type': 'softmax'}, 'batch_size': 1, 'eval_batch_size': 1, 'learning_rate': 0.001, 'kernel_reg': 0.0, 'detection_threshold': 0.001, 'nms': 4, 'dense_loss': {'enable': False, 'params': {'descriptor_dist': 4, 'lambda_d': 800}}, 'sparse_loss': {'enable': True, 'params': {'num_matching_attempts': 1000, 'num_masked_non_matches_per_match': 100, 'lamda_d': 1, 'dist': 'cos', 'method': '2d'}}, 'other_settings': 'train 2d, gauss 0.5'}, 'data': {'dataset': 'CubemapDataset', 'primitives': 'all', 'not_warped_images': True, 'preprocessing': {'resize': [1024, 1024]}}, 'front_end_model': 'Train_model_frontend_cubemap', 'retrain': False, 'reset_iter': True, 'validation_inte

2023-04-10 16:23:19 koala-A520M-H root[25190] INFO => setting adam solver
2023-04-10 16:23:19 koala-A520M-H root[25190] INFO reset iterations to 0


adam optimizer
load pretrained model from: %s logs/magicpoint_synth_t2/checkpoints/3channel.pth.tar
successfully load pretrained model from: %s logs/magicpoint_synth_t2/checkpoints/3channel.pth.tar


 Train only descriptor
=== Let's use 1 GPUs!
adam optimizer


In [14]:
a = torch.Tensor([1])[0]
b = torch.Tensor([2])[0]
print(a,b)
for i in range(2):
    b += a
    
b

tensor(1.) tensor(2.)


tensor(4.)

In [10]:
a, b

(tensor([1.]), tensor(3.))

In [5]:
sample =next(iter(train_loader))
net = dc(train_agent.net)

In [6]:
sample.keys(), sample['image'].shape

(dict_keys(['image', 'warped_image', 'ply_path', 'R', 'T', 'R_w', 'T_w', 'img_path', 'img_path_w', 'kpts2D', 'kpts3D', 'kpts2D_w', 'kpts3D_w']),
 torch.Size([1, 3, 1024, 1024]))

In [7]:

img, img_w = sample['image'].to(device), sample['warped_image'].to(device)


In [14]:
img_shape = (3, 4, 4)
labels_2D = torch.Tensor([[0,0], [1,1], [3,1], [2,1]])
background = torch.zeros(*img_shape).to(device)
background[:, labels_2D[:,0].int(), labels_2D[:,1].int()] = 1
background

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.]],

        [[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.]],

        [[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.]]], device='cuda:0')

In [513]:
samim = torch.Tensor(np.array(Image.open(imname2).resize((1024,1024))))
samim = samim.transpose(1,2).transpose(0,1)

In [21]:
hm = torch.zeros(3, 1, 4, 4)
hm[0,0, 1,1] = 1
hm[1,0, 1,2] = 1
hm[2,0, 3,3] = 1
rs, cs = torch.where(hm.reshape(12, 4) > 0)
kpts = torch.stack((
    cs, rs), dim=1)
kpts = kpts.tolist()
b_kpts = [[]]*3
b_kpts

tensor([[1, 1],
        [2, 1],
        [3, 3]])

In [22]:
hm.shape

torch.Size([3, 1, 4, 4])

In [24]:
hm.reshape(12, 4)

tensor([[0., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 1.]])

In [25]:
a = [1,2,3]
int(a)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [8]:
HOMO_NUM, HOMO_BATCH = 100, 20
batch_size = img.shape[0]
processed_img_names = []
import time
thd=0.2
fnc = 1023.5
camera_matrix = torch.Tensor([[fnc,0,fnc],[0,fnc,fnc],[0,0,1]])

kpts2d_total, kpts2d_total_w = [[]]*batch_size, [[]]*batch_size
kpts3d_total, kpts3d_total_w = [[]]*batch_size, [[]]*batch_size
start = time.time()

ptcloud_list = []
for b in range(batch_size):
    target = o3d.io.read_point_cloud(sample['ply_path'][b])
    points = torch.Tensor(np.array(target.points)).to(device)
    del target
    ptcloud_list.append(points)


In [ ]:
# 가장 내적값이 큰 kpt의 descriptor를 연결
out['desc'].shape

In [ ]:


desc = train_agent.interpolate_to_dense(out['desc'])
desc_w = train_agent.interpolate_to_dense(out_w['desc'])
thd = 0.2
# hms = thd_img(flattenDetection(out['semi']), thd=thd)
# hms_w = thd_img(flattenDetection(out_w['semi']), thd=thd)
hms = flattenDetection(out['semi'])
hms_w = flattenDetection(out_w['semi'])
print(desc.shape)
b = 0
kpts = get_kpts_from_hm(hms[b])
kpts_w = get_kpts_from_hm(hms_w[b])
print("Num of kpts: ", kpts.shape, kpts_w.shape)

kpts_desc = get_desc_of_kpts(desc[b], kpts)
kpts_desc_w = get_desc_of_kpts(desc_w[b], kpts_w)
kpts_desc = kpts_desc.transpose(0, 1)
kpts_desc_w = kpts_desc_w.transpose(0, 1)

In [ ]:
kpts_desc.shape, kpts_desc_w.shape

In [ ]:
# a = torch.Tensor([[0,0],   [0,1],   [0,2],   [0,3],   [0,4]])
# b = torch.Tensor([[0,0.2], [0,3.2], [0,3.1], [0,1.1], [0,4.1]])
# '''
# GT: 0 2 3 nan 4
# Return: [0,1,3,4], [0,3,2,4]
# '''

# # a = torch.Tensor([[0,5],   [0,1],   [0,6],   [0,3],   [0,4]])
# # b = torch.Tensor([[0,4], [0,5.51], [0,3.1], [0,1.1], [0,4.1]])
# '''
# Return: [1,2, 3, 4], [3,1,2,0]
# '''

# sort_by_nearest_pts(a, b)

In [ ]:

def sort_by_nearest_pts(kpts1, kpts2):
    # sort kpts2 correspondent to kpts1
    ary = torch.cdist(kpts2, kpts1)
    mask1 = ary == torch.unsqueeze(torch.min(ary,axis=1).values, dim=1)
    mask2 = ary == torch.min(ary, axis=0).values
    idx1, idx2 = torch.where(mask1*mask2*(ary<10))
    return kpts1[idx2], kpts2[idx1], ary[idx1, idx2]

In [ ]:
# kpts1, kpts2 = kpts.float(), kpts_w.float()
# ary = torch.cdist(kpts2, kpts1)
# mask1 = ary == torch.unsqueeze(torch.min(ary,axis=1).values, dim=1)
# mask2 = ary == torch.min(ary, axis=0).values
# idx1, idx2 = torch.where(mask1*mask2*(ary<5))
# ary[idx1, idx2]

In [ ]:
b = 0

In [ ]:
kpts_sorted, kpts_w_sorted, color = sort_by_nearest_pts(kpts.float(), kpts_w.float())
kpts_w_sorted.shape, max(color), min(color)

In [ ]:
color = 255-color*10
color = torch.clamp(color, 0, 255)
color = torch.stack([torch.zeros_like(color), color, color], axis=1)
color.shape

In [ ]:
img[0].shape

In [ ]:

outfig = make_matching_plot_fast(sample['img_path'][0], sample['img_path_w'][0], 
        kpts.detach().cpu().numpy(), kpts_w.detach().cpu().numpy(), 
        kpts_sorted.detach().cpu().numpy(), kpts_w_sorted.detach().cpu().numpy(), color,
                                 img_shape=img.shape[2:])
plt.imshow(outfig)
plt.savefig('rd_glue2.jpg', dpi=1200)

In [42]:
a = torch.ones(1,8,8)
Hinfo = {'theta': 22.903534447864008, 't_x': -0.984204265504758, 't_y': 0.32934251819071, 'scale': 0.7376072030495775, 'startpoints': [[1.369664350014332, -1.067612001895284], [-1.088877579074605, 1.8467378868434], [7.2300664433097, 7.6867897737297], [7.757960087271, 7.527099415175414]], 'endpoints': [[   0.,    0.],
       [   0., 8.],
       [8., 8.],
       [8.,    0.]]}
a = apply_H_from_info(a, Hinfo)
a[0]

tensor([[1.0000, 0.9643, 0.0704, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 0.1580, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 0.2231, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 0.2669, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 0.2906, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 0.2951, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 0.2881, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 0.3396, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

In [45]:
str(type(None))

"<class 'NoneType'>"

In [44]:
def rmedge(p4list, px):
    new = []
    for p in p4list:
        e1, e2 = -px, -px
        if p[0] == 0:
            e1 *= -1
        if p[1] == 0:
            e2 *= -1
        new.append([p[0]+e1, p[1]+e2])
    return new

rmedge(Hinfo['endpoints'], 2)

[[2.0, 2.0], [2.0, 6.0], [6.0, 6.0], [6.0, 2.0]]

In [ ]:
thd=0.4
out = net(img)
hms = thd_img(flattenDetection(out['semi']), thd=thd)
b=0
hm = hms[b][0]
rs, cs = torch.where(hm > 0)
kpts = torch.stack((cs, rs), dim=1)
print(len(kpts))
showkpts(img[b].transpose(0,1).transpose(1,2).detach().cpu(), kpts)

In [ ]:
uniq = np.unique(kpts_total[0], axis=0)
uniq.shape

In [ ]:
thd=0.4
img, img_w = sample['image'].to(device), sample['warped_image'].to(device)

kpts_total = [[]]*2
for homo_iter in range(3):
    im_transformed, Hinv_info = apply_random_H_batch(img)
    im_transformed_w, Hinv_info_w = apply_random_H_batch(img_w)
    out = net(im_transformed)
    out_w = net(im_transformed_w)

    hms = flattenDetection(out['semi'])
    im_orig = apply_H_from_info(hms, Hinv_info)
    im_orig = thd_img(im_orig, thd=thd)

    hms_w = flattenDetection(out_w['semi'])
    im_orig_w = apply_H_from_info(hms_w, Hinv_info_w)
    im_orig_w = thd_img(im_orig_w, thd=thd)

    print(torch.sum(im_orig[0]), im_orig.shape)

    for b in range(2):
        hm, hm_w = im_orig[b][0], hms_w[b][0]
        rs, cs = torch.where(hm > 0)
        rs_w, cs_w = torch.where(hm_w > 0)
        kpts = torch.stack((cs, rs), dim=1)
        kpts_total[b]+=kpts.detach().cpu().tolist()
b=0
kpttotal = np.unique(kpts_total[0], axis=0)
print(kpttotal.shape)
showkpts(img[b].transpose(0,1).transpose(1,2).detach().cpu(), kpttotal)

In [ ]:
im_orig[0,:,:].shape

In [ ]:
torch.topk(torch.flatten(torch.where(im_orig[0,:,:]>0.2, diff, 0)), 3)

In [ ]:
hms.shape

In [ ]:
# inputim = dc(imsam[:1])
hms = flattenDetection(out['semi'])
inputim = dc(hms[0])
thd = 0.2
showim(thd_img(inputim, thd=thd))

im_transformed, Hinv_info = apply_random_H(inputim)
im_orig = apply_H_from_info(im_transformed, Hinv_info)
# im_orig = thd_img(im_orig, thd=0.5)

In [ ]:

diff = thd_img(inputim, thd=thd)-thd_img(im_orig, thd=thd)
showim(diff)
showim(thd_img(im_orig, thd=thd))

In [ ]:
I = thd_img(im_orig, thd=thd)
torch.sum(I)